Установим библиотеку для раскрытия кратких форм в английском языке (I`m -> I am)

In [ ]:
!pip install contractions -q

## Импортируем библиотеки 

In [ ]:
import html
import gdown
import pandas as pd
import contractions

## Выгрузим исходные данные

In [ ]:
df_train = pd.read_json('ranking_train.jsonl', lines=True)
df_test = pd.read_json('ranking_test.jsonl', lines=True)

df_train.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88107 entries, 0 to 88106
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      88107 non-null  object
 1   comments  88107 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14004 entries, 0 to 14003
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      14004 non-null  object
 1   comments  14004 non-null  object
dtypes: object(2)
memory usage: 218.9+ KB


Шаги предобработки данных:
1. Создается регулярное выражение pattern_post_text для замены url в текстах постов.
2. Создается регулярное выражение pattern_comm_text для замены url в текстах комментариев.
3. Создается копия переданного DataFrame df.
4. DataFrame df "разворачивается", то есть каждый элемент в столбце комментариев становится отдельной строкой.
5. Из столбца комментариев создаются два столбца: comment_text и comment_score.
6. Столбец комментариев удаляется из DataFrame df.
7. Столбцы переименовываются: index -> post_index, text -> post_text.
8. Тексты постов и комментариев декодируются из HTML-сущностей и исправляются апострофы.
9. Тексты постов и комментариев приводятся к нижнему регистру, заменяются множественные пробелы на один, а ссылки заменяются на слово "url".
10. В DataFrame добавляются два новых столбца: post_text_fix и comment_text_fix, которые содержат предобработанные тексты постов и комментариев соответственно.
11. DataFrame сохраняется в файл с помощью функции to_feather(), используя переданный путь filepath.

In [ ]:
def prep_dataframes(df, filepath):
    """
    Функция prep_dataframes() выполняет предобработку данных в переданном DataFrame. 

    Аргументы функции:
    - df: DataFrame, который нужно предобработать
    - filepath: строка с путем для сохранения файла с предобработанными данными

    Результат:
    - функция сохраняет DataFrame с предобработанными данными в формате feather, используя переданный путь filepath.
    """

    pattern_post_text = r"""(?:https?://|www\.)\S+\b(?!.*\?(?=[^\s\)\]]))/?"""
    pattern_comm_text = r"""(?:https?://|www\.)\S+?(?=\[|$|\s)"""

    df = df.copy()
    df = df.explode('comments').reset_index()
    df[['comment_text', 'comment_score']] = \
        pd.DataFrame.from_records(df['comments'].values)
    df = df.drop('comments', axis=1)
    df = df.rename(columns={'index': 'post_index', 'text': 'post_text'})
    df[['post_text', 'comment_text']] = (
        df[['post_text', 'comment_text']]
        .applymap(html.unescape)
        .applymap(contractions.fix))
    df['post_text_fix'] = df['post_text'].str.lower().str.replace(r"\s+", ' ', regex=True).str.replace(pattern_post_text, 'url', regex=True)
    df['comment_text_fix'] = df['comment_text'].str.lower().str.replace(r"\s+", ' ', regex=True).str.replace(pattern_comm_text, 'url', regex=True)
    df.to_feather(filepath)

prep_dataframes(df_train, 'df_train_BERT.feather')
prep_dataframes(df_test, 'df_test_BERT.feather')